In [1]:
# Create an individual. An individual is a member of a population.

from random import randint
import random

def individual(length, min, max):
    return [randint(min,max) for x in range(length)]
 
# print(individual(5,0,100))
# print(individual(5,0,100))



In [2]:
def population(count, length, min, max):
     return [individual(length, min, max) for x in range(count)]

population(3,5,0,100)

[[5, 100, 32, 64, 42], [21, 66, 24, 76, 4], [59, 82, 86, 21, 94]]

In [3]:
def fitness(individual,target):
    total = sum(individual)
    return abs(target-total)

x = individual(5,0,100)
# print(x)
fitness(x, 200)

40

In [4]:
def grade(pop,target):
    summed = [sum(i)-target for i in pop]
    # print(summed)
    return (sum(summed) / len(pop))
    
    
    
x = population(3,5,0,100)
# print(x)
target = 200
grade(x, target)


38.333333333333336

In [5]:
def evolve(pop, target, retain, random_select, mutate):
    graded = [(fitness(x, target), x) for x in pop]
    graded = [x[1] for x in sorted(graded)] # x[1] because x has two components, just take the list -->e.g. [(50, [41, 38, 86, 30, 55])]
    retain_length = int(len(graded)*retain) # how many top % parents to be remained.
    parents = graded[0:retain_length] # get the list of array of individuals as parents - after sorted
    
    
 # randomly add other individuals to promote genetic diversity
    for individual in graded[retain_length:]: # get from remaining individuals NOT selected as parents initially!
        if random_select > random.random():
            parents.append(individual)

    # mutate some individuals
    for individual in parents:
        if mutate > random.random():
            pos_to_mutate = randint(0, len(individual)-1)
            # this mutation is not ideal, because it restricts the range of possible values, but the function is unaware of the min/max values used to create the individuals
            individual[pos_to_mutate] = randint(min(individual), max(individual))
            # individual[pos_to_mutate] = toolbox.register("individual",make_individual,explanatories=explanatories)
   
    #crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length # assuming that you need to make sure you have enough individuals
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = int(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)
    
    parents.extend(children)
    return parents

In [38]:
import random
import pygad
import numpy
from tabulate import tabulate
from deap import base, creator, tools
from typing import Optional, List, Any


creator.create("FitnessMin1", base.Fitness, weights=(-1.0,))   
creator.create("Individual", list, fitness=creator.FitnessMin1)   # individual data-type : List
toolbox = base.Toolbox()


'''
Functions
'''

def random_pick(lower: int, upper: int) -> int:
    # Pick random numbers within specified bounds 
        return randint(lower, upper)


def make_individual(explanatories) -> List[Any]:
     individual = []
     for explanatory in explanatories:
         individual.append(
             random_pick(
                 lower=explanatories[explanatory][0],
                 upper=explanatories[explanatory][1],
                 # step=explanatories[explanatory][2] if len(explanatories[explanatory]) == 3 else None,
             )
         )
     return creator.Individual(individual)


'''
Main
'''
pop_size = 100
Total_money = 5000
vacation_duration =5
money = Total_money/vacation_duration

touristSpot = 10
transport_freq = 5

# Hotelprice=int((((money*30)/100)/vacation_duration))
# FoodPrice=int((((money*25)/100)/(vacation_duration*3))) #one day 3meals
# OneTouristSpot=int((((money*25)/100)/touristSpot)) 
# Transportationfees=int((((money*20)/100)/25))

Hotelprice=int((((money*30)/100)))
FoodPrice=int((((money*15)/100)))
OneTouristSpot=int((((money*35)/100)))
Transportationfees=int((((money*20)/100)))


# low=[0, 0, 0, 0] #lower bound for each of the ranges
# high=[Hotelprice,FoodPrice,OneTouristSpot,Transportationfees] #upper bound for each of the ranges

explanatories={
    "a": (100, Hotelprice),   # (low, high, step)
    "b": (10, FoodPrice),
    "c": (20, OneTouristSpot),
    "d": (5, Transportationfees),
    # "e": (15, 200, 1),
    # "f": (5, 1699, 1),
    # "g": (0.5, 0.5),     # (low, high)
    }

toolbox.register(
  "individual", 
  make_individual,
  explanatories=explanatories  # now toolbox.individual uses default explanatories
)
ind = toolbox.individual()  # no longer provide explanatories arg
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
p = toolbox.population(pop_size)
# print(int)
# print(p)

value_lst =[]
fitness_history = []

target = Total_money/vacation_duration
i_length = 4
i_min = 0
i_max = 1000
n_generation = 100
c =0
for x in range(3):
    c = c+1
    if c==1:
        retain=0.1
        random_select=0.02
        mutate=0.01
    if c==2:
        retain=0.5
        random_select=0.09
        mutate=0.05
    if c==3:
        retain=0.8
        random_select=0.10
        mutate=0.12
        
    for i in range(n_generation):
        p = evolve(p, target, retain ,random_select, mutate)
        value = grade(p, target)
        # if value >=-1 and value <=1 :
        #     break
        fitness_history.append(value)
        value_lst.append(p[0])
        value_lst.append(value)

    # for datum in fitness_history:
    #     print(datum)
    print('Retain:',retain) 
    print('Random Select:',random_select) 
    print('Mutation:',mutate)
    last_element = fitness_history[-1]
    print('Fitness :',last_element)

    # value_lst

    hotelstar = int((value_lst[pop_size][0]))
    TouristSpot1 = int(((value_lst[pop_size][2]*50)/100))
    Foodprice1= int((value_lst[pop_size][1]/3))
    Transportationfees1 = (value_lst[pop_size][3]/transport_freq)

    HotelstarRating = '<RM' + str(hotelstar) + ' per night'
    TouristSpotFinal = str(touristSpot) + ' spots'
    OneTouristSpot = '<RM' + str(TouristSpot1)
    FoodPriceFinal = '<RM' + str(Foodprice1) + ' per meal'
    TransportationFee = '<RM' + str(Transportationfees1) + ' per trip'
    TransportFrequency = str(transport_freq) + ' trip per day'
    # #create data
    data = [["Money on hand", (target*vacation_duration)], 
            ["Vacation Duration", vacation_duration], 
            ["Hotel Star rating",  HotelstarRating], 
            ["Tourist Spot", TouristSpotFinal],
            ["One Tourist Spot", OneTouristSpot],
            ["Food Price", FoodPriceFinal],
            ["Transportation fees", TransportationFee],
            ["Transport Frequency", TransportFrequency]]

    #define header names
    col_names = ["Parameter", "Values"]

    #display table
    print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))


Retain: 0.1
Random Select: 0.02
Mutation: 0.01
Fitness : 47.0
╒═════════════════════╤══════════════════╕
│ Parameter           │ Values           │
╞═════════════════════╪══════════════════╡
│ Money on hand       │ 5000.0           │
├─────────────────────┼──────────────────┤
│ Vacation Duration   │ 5                │
├─────────────────────┼──────────────────┤
│ Hotel Star rating   │ <RM283 per night │
├─────────────────────┼──────────────────┤
│ Tourist Spot        │ 10 spots         │
├─────────────────────┼──────────────────┤
│ One Tourist Spot    │ <RM167           │
├─────────────────────┼──────────────────┤
│ Food Price          │ <RM47 per meal   │
├─────────────────────┼──────────────────┤
│ Transportation fees │ <RM37.4 per trip │
├─────────────────────┼──────────────────┤
│ Transport Frequency │ 5 trip per day   │
╘═════════════════════╧══════════════════╛
Retain: 0.5
Random Select: 0.09
Mutation: 0.05
Fitness : 0.76
╒═════════════════════╤══════════════════╕
│ Parameter     